In [376]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import math

In [377]:
win = 1
loss = -1
draw = -1

rows = 3
cols = 3

x = 1
o = 2
empty = 0


In [378]:
class TicTacToeNetworkA(nn.Module):
    def __init__(self):
        super(TicTacToeNetworkA, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(81, 8000),
            nn.ReLU(),
            nn.Linear(8000, 800),
            nn.ReLU(),
            nn.Linear(800, 9)
        )

    def forward(self, x):
        x = x.unsqueeze(0)
        x = self.flatten(x)
        x = self.layers(x)
        return x
    
modelA = TicTacToeNetworkA()
optimizerA = optim.Adam(modelA.parameters(), lr=0.001)
criterionA = nn.MSELoss()

In [379]:
def user(game):
    while True:
        row = int(input(f'Enter row (0-{rows-1}): '))
        if row < rows and row >= 0:
            break

    while True:
        col = int(input(f'Enter col (0-{cols-1}): '))
        if col < cols and col >= 0:
            break

    board = torch.ones(rows,cols)
    board = board * empty

    board[row][col] = 1

    return board.view(1,9)

In [380]:
def checkForWin(board):
    for row in board:
        if torch.all(row == x):
            return x
        elif torch.all(row == o):
            return o

    for col in range(0, cols):
        col = board[:,col]
        if torch.all(col == x):
            return x
        elif torch.all(col == o):
            return o
        
    truth = board == o
    if truth[0][0] and truth[1][1] and truth[2][2]:
        return o
    elif truth[0][2] and truth[1][1] and truth[2][0]:
        return o
    
    truth = board == x
    if truth[0][0] and truth[1][1] and truth[2][2]:
        return x
    elif truth[0][2] and truth[1][1] and truth[2][0]:
        return x
        
    return empty

def print_tic_tac_toe_board(board):
    for row in range(3):
        # Print horizontal lines between rows
        if row != 0:
            print("-" * cols  * 3)

        for col in range(3):
            # Print vertical lines between columns
            if col != 0:
                print("|", end=" ")

            # Print the cell value
            if board[row][col] == 0:
                print(" ", end=" ")
            elif board[row][col] == 1:
                print("X", end=" ")
            elif board[row][col] == 2:
                print("O", end=" ")

        print()  # Move to the next line after printing each row
    print()



def run_game(playerA, playerB):
    game = torch.ones(rows*cols, rows,cols)
    game = game * empty

    for i in range(0, rows * cols):
        icon = empty
        
        if (i % 2) == 0:
            move = playerA(game)
            icon = x
        else:
            move = playerB(game)
            icon = o

        row, col = interprateMove(move)

        if game[i][row][col] == empty:
            for j in range(i,rows*cols):
                game[j][row][col] = icon


        print_tic_tac_toe_board(game[i])

        if checkForWin(game[i]):
            return checkForWin(game[i])
        
    return checkForWin(game[i]), game, move 


def interprateMove(move):
    probs = F.softmax(move, dim=1)
    flat_indices = torch.argmax(probs.view(-1))
    original_indices = torch.unravel_index(flat_indices, move.shape)
    _, flat_index = original_indices
    row = math.floor(flat_index / rows)
    col = (flat_index % rows).item()
    return row, col




In [ ]:
winner = run_game(modelA, modelA)

loss = criterionA()

In [382]:
winner = run_game(user, modelA)

print(f"player {winner} is the winner!")



  |   |   
---------
X |   |   
---------
  |   |   

O |   |   
---------
X |   |   
---------
  |   |   

O |   |   
---------
X | X |   
---------
  |   |   

O |   |   
---------
X | X |   
---------
  |   |   

O |   |   
---------
X | X | X 
---------
  |   |   

player 1 is the winner!
